<a href="https://colab.research.google.com/github/futartup/S11-assignment/blob/master/model/s11_resnet_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **This is a model implementation of assignment s11, with resnet block**

In [0]:
import torch
import torch.nn as nn
from torchsummary import summary
import torch.nn.functional as F

Return Conv2d layer

In [0]:
def conv2d(in_channels=0, out_channels=0, filter_size=1, stride=1, padding=0, bias=False ):
  return nn.Conv2d(in_channels, out_channels, filter_size, stride, padding, bias)

Return Batch Normalization Layer

In [0]:
def batchN(in_channels=1):
  return nn.BatchNorm2d(in_channels)

Return Max Pool Layer

In [0]:
def maxpool(x=1,y=1):
  return nn.MaxPool2d(x, y)

Fully Connected Layer

In [0]:
def fc(in_channels=1, out_channels=1):
  return nn.Linear(in_channels, out_channels)

Relu

In [0]:
def relu():
  return nn.ReLU()

Resnet Block

In [0]:
class ResnetBlock(nn.Module):
  def __init__(self, topology):
    super(ResnetBlock, self).__init__()
    for key, val in topology.items():
      self.r = self.make_layers(topology)

  def make_layers(self, topology: dict):
    layers = []
    assert len(topology) > 0, "The resnet block should contain some blocks"
    for key, val in topology.items():
      for key, val in val.items():
        layers.append(topology_mapping[key](**val))
    return nn.Sequential(*layers)

  def forward(self, x):
    initial = x
    for l in self.__dir__['_modules']:
      out = l(x)
      x = out
    x += initial
    return x

Topology Mapping

In [0]:
topology_mapping = {
    'C': conv2d,
    'BN': batchN,
    'MP': maxpool,
    'FC': fc,
    'Rel': relu,
    'R': ResnetBlock
}

Lets Go

In [0]:
topology = {
    "preparation_layer": {"C":{"in_channels": 3,"out_channels": 64,"filter_size": 3,"stride": 1,"padding": 1, "bias": False,},
                          "BN": { "in_channels": 64 },
                          "Rel": {},
                          },

    "layer1": {
        "C": {"in_channels": 64,"out_channels": 128,"filter_size": 3,"stride": 1,"padding": 1, "bias": False,},      
        "MP": {  "x": 2, "y": 2 },
        "BN": { "in_channels": 128 },
        "Rel": {},
        "R": {
            "b1":{
                "C": {"in_channels": 128,"out_channels": 128,"filter_size": 3,"stride": 1,"padding": 1 , "bias": False,},
                "BN": { "in_channels": 128 },
                "Rel": {},
            },
            "b2":{
                "C": {"in_channels": 128,"out_channels": 128,"filter_size": 3,"stride": 1,"padding": 1 , "bias": False,},
                "BN": { "in_channels": 128 },
                "Rel": {},
            },
            
        }
    },

    "layer2": {
        "C": { "in_channels": 128,"out_channels": 256,"filter_size": 3,"stride": 1,"padding": 1, "bias": False,},      
        "MP": {   "x": 2, "y": 2},
        "BN": {  "in_channels": 256 },
        "Rel": {},
    },

    "layer3": {
        "C": { "in_channels": 256,"out_channels": 512,"filter_size": 3,"stride": 1,"padding": 1, "bias": False,},      
        "MP": {   "x": 2, "y": 2 },
        "BN": {  "in_channels": 512 },
        "Rel": {},
        "R": {
            "b1": {
                  "C": { "in_channels": 512,"out_channels": 512,"filter_size": 3,"stride": 1,"padding": 1, "bias": False, },
                  "BN": {   "in_channels": 512 },
                  "Rel": {},
            },
            "b2":{
                 "C": { "in_channels": 512,"out_channels": 512,"filter_size": 3,"stride": 1,"padding": 1 , "bias": False,},
                 "BN": {   "in_channels": 512 },
                 "Rel": {},
            }
           
        }
    },
    "maxpool_4": {
        "MP": {   "x": 4, "y": 4},
    },
    "FC": {
        "FC": { "in_channels": 512, "out_channels": 10},
    },
}


class S11(nn.Module):
  def __init__(self, topology):
    super(S11, self).__init__()
    for key, val in topology.items():
      self.__dict__['_modules'][key] = self.make_layer(val)

  def make_layer(self, topology):
    layers = []
    for key, val in topology.items():
      if key == 'R':
        layers.append(ResnetBlock(val))
      else:
        layers.append(topology_mapping[key](**val))
    return nn.Sequential(*layers)

  def forward(self, x):
    x = self.preparation_layer(x)
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.maxpool_4(x)
    x = torch.flatten(x, 1)
    x = self.FC(x)
    #x = x.view(-1, 10)
    x = F.log_softmax(x, dim=1)
    return x

Define the Network Architecture

In [0]:
#net = S11(topology)


# use_cuda = torch.cuda.is_available()
# if use_cuda:
#   torch.cuda.manual_seed(1)
# device = torch.device("cuda" if use_cuda else "cpu")
# print(device)
# model = net.to(device)
# summary(model, input_size=(3, 32, 32))

In [0]:
class NetS11(nn.Module):
  def __init__(self):
    super(NetS11, self).__init__()

    self.preparation_layer = nn.Sequential(
        nn.Conv2d(3, 64, 3, padding=1, stride=1, bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU()
    )

    self.layer1 = nn.Sequential(
        nn.Conv2d(64, 128, 3, padding=1, stride=1, bias=False),
        nn.MaxPool2d(2, 2),
        nn.BatchNorm2d(128),
        nn.ReLU()
    )

    self.layer1_resnet = nn.Sequential(
        nn.Conv2d(128, 128, 3, padding=1, bias=False),
        nn.BatchNorm2d(128),
        nn.ReLU(),

        nn.Conv2d(128, 128, 3, padding=1, bias=False),
        nn.BatchNorm2d(128),
        nn.ReLU()
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(128, 256, 3, padding=1, bias=False),
        nn.MaxPool2d(2, 2),
        nn.BatchNorm2d(256),
        nn.ReLU()
    )

    self.layer3 = nn.Sequential(
        nn.Conv2d(256, 512, 3, padding=1, stride=1, bias=False),
        nn.MaxPool2d(2, 2),
        nn.BatchNorm2d(512),
        nn.ReLU()
    )

    self.layer3_resnet = nn.Sequential(
        nn.Conv2d(512, 512, 3, padding=1, bias=False),
        nn.BatchNorm2d(512),
        nn.ReLU(),

        nn.Conv2d(512, 512, 3, padding=1, bias=False),
        nn.BatchNorm2d(512),
        nn.ReLU()
    )

    self.maxpool_k = nn.MaxPool2d(4,4)

    self.linear = nn.Linear(512, 10)


  def forward(self,x):
    x = self.preparation_layer(x)
    
    x = self.layer1(x)
    initial = x
    x = self.layer1_resnet(x) + initial
    
    x = self.layer2(x)

    x = self.layer3(x)
    initial = x
    x = self.layer3_resnet(x) + initial
    
    x = self.maxpool_k(x)
    
    x = torch.flatten(x, 1)
    x = self.linear(x)
    
    x = F.log_softmax(x, dim=1)
    return x